In [1]:
import pandas as pd
import multiprocessing
import json
import time
import os
import re

In [2]:
# Slice large file to plenty of smaller ones

def slicer(input_path_to_file, output_file_path, output_lines):
    out = []
    i = 0
    with open(input_path_to_file) as file:
        for line in file:
            out.append(line)
            if len(out) > output_lines:
                print(i)
                with open(output_file_path.format(i), mode='a') as file2:
                    file2.write(json.dumps(out))
                    i += 1
                    out = []
        if len(out) != 0:
            with open(output_file_path.format(i), mode='a') as file2:
                file2.write(json.dumps(out))


In [3]:
def json_to_csv(file_name):
    print('\t{}'.format(file_name))
    # Process data
    # Please notice that we using same name of variable to decrease memory usage

    with open(file_name) as f:
        file = json.load(f)

    # Unpack json rows
    file = [json.loads(x) for x in file]

    file = pd.DataFrame(file)

    # Remove unnecessary columns
    file = file.filter(['overall', 'reviewText'])

    # Rename columns
    file = file.rename(columns={'overall': 'sentiment', 'reviewText': 'text'})

    # Transform 0-5 score, into positive/negative values

    file = file[file.sentiment != 3]
    file.sentiment = file.sentiment.apply(lambda x: 'positive' if x > 3 else 'negative')

    # Remove noises
    # Make all to string
    file['text'] = file['text'].apply(str)

    # We can remove some duplicates also at this point
    file = file.drop_duplicates()

    patterns = [
        r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
        r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
        r"(@[A-Za-z0-9_]+)",
        r"(#[A-Za-z0-9_]+)"]

    for pattern in patterns:
        file['text'] = file['text'].apply(lambda x: re.sub(pattern, '', str(x)))

    file.text = file.text.apply(lambda x: x if 0 < len(str(x).split()) < 100 else None)

    # Remove nones
    file = file.dropna()

    # Write into csv file
    file.to_csv(file_name, mode='w', index=False)

In [4]:
class ReSizerFiles(object):
    def __init__(self):
        super(ReSizerFiles, self).__init__()

    @staticmethod
    def re_sizer(input_file_name: str, output_file_name: str,
                 max_length_of_single_output: int, max_amount_output_of_files=float('inf'),
                 total_length_of_output: int = float('inf'),
                 auto_file_remover: bool = False, auto_detect_files_in_dir: bool = False, files_in_folder: int = None):

        if not auto_detect_files_in_dir and not files_in_folder:
            raise Exception("Declare files_in_folder or auto_detect_files_in_dir")

        if auto_detect_files_in_dir and not files_in_folder:
            files_in_folder = [x for x in os.listdir(os.path.split(input_file_name)[0]) if
                               os.path.split(input_file_name)[-1].replace("{", '').replace("}", '') in x
                               and os.path.split(input_file_name)[-1].replace("{", '').replace("}", '') ==
                               re.sub(r'\d', '', x)].__len__()

        files_name_lest_to_remove = [input_file_name.format(x) for x in range(files_in_folder)]

        index_output_file = 0
        df_out = pd.DataFrame()
        df = pd.DataFrame()

        # Iterate over each file declared as input

        for i in range(files_in_folder):

            if files_in_folder == 0:
                df.drop(df.index, inplace=True)
                df_out.drop(df_out.index, inplace=True)
                # Break after break in while loop
                break

            if i == 21:
                print(123)
            print("input_file_{}| output_file_{}".format(i, index_output_file))

            df = pd.read_csv(input_file_name.format(i))

            length = df_out.__len__() + df.__len__()

            # Check if file is enough large to save it
            if length >= max_length_of_single_output or total_length_of_output <= max_length_of_single_output:
                # Calculate left where to cut data frame to get equal value to declared one
                to_cut = df.__len__() - (length - max_length_of_single_output)

                if total_length_of_output < max_length_of_single_output:
                    to_cut = total_length_of_output
                    if to_cut == 0:
                        df.drop(df.index, inplace=True)
                        df_out.drop(df_out.index, inplace=True)
                        break
                    if length - max_length_of_single_output < 0:
                        to_cut = total_length_of_output - df_out.__len__()

                df_out = pd.concat([df_out, df[:to_cut]])

                df_out.to_csv(output_file_name.format(index_output_file), mode='a', index=False)
                total_length_of_output -= df_out.__len__()

                # Overwrite file data frame and remove already saved data
                df_out = pd.concat([df[to_cut:]])
                df.drop(df.index, inplace=True)

                if max_amount_output_of_files == index_output_file \
                        or total_length_of_output <= 0:
                    df.drop(df.index, inplace=True)
                    df_out.drop(df_out.index, inplace=True)
                    # remove loop if get declared amount of file in output
                    break

                index_output_file += 1

                # Loop for deal with larger file separated to few smaller one
                while True:
                    # Is size of df_positive is much larger than limit
                    if df_out.__len__() > max_length_of_single_output:
                        print("input_file_{}| output_file_{}".format(i, index_output_file))
                        if total_length_of_output < max_length_of_single_output:
                            max_length_of_single_output = total_length_of_output

                        df_out[:max_length_of_single_output].to_csv(output_file_name.format(index_output_file), mode='a',
                                                                    index=False)

                        total_length_of_output -= df_out[:max_length_of_single_output].__len__()

                        df_out = df_out[max_length_of_single_output:]

                        if max_amount_output_of_files <= index_output_file or \
                                total_length_of_output == 0:
                            # remove loop if get declared amount of file in output
                            files_in_folder = 0
                            break

                        index_output_file += 1


                    else:
                        break
            else:
                df_out = pd.concat([df_out, df])
                df.drop(df.index, inplace=True)

            if auto_file_remover:
                os.remove(input_file_name.format(i))
                files_name_lest_to_remove.remove(input_file_name.format(i))

        if df_out.__len__() != 0:
            df_out.to_csv(output_file_name.format(index_output_file), mode='a', index=False)

        if auto_file_remover:
            for file_to_remove in files_name_lest_to_remove:
                os.remove(file_to_remove)


In [5]:
def duplicate_remover(input_file_name,
                      output_file_name_neg,
                      output_file_name_pos,
                      files_in_folder,
                      auto_remove_files=False):
    # Drop duplicates
    list_files_1 = [x for x in range(files_in_folder)]
    list_files_2 = [x for x in range(files_in_folder)]

    for i in list_files_1:
        print('duplicate_remover {}'.format(i))
        list_files_2.remove(i)
        df1 = pd.read_csv(input_file_name.format(i))
        # Remove duplicated rows in data frame
        df1 = df1.drop_duplicates()

        # Prepare positive list of sentences and filter them
        pos1 = set(df1[df1.sentiment == 'positive']['text'])
        pos1 = set(filter(lambda x: type(x) == str, pos1))

        # Prepare negative list of sentences and filter them
        neg1 = set(df1[df1.sentiment == 'negative']['text'])
        neg1 = set(filter(lambda x: type(x) == str, neg1))

        for i2 in list_files_2:
            print('\t{}'.format(i2))
            df2 = pd.read_csv(input_file_name.format(i2))
            df2 = df2.drop_duplicates()
            pos2 = set(df2[df2.sentiment == 'positive']['text'])
            neg2 = set(df2[df2.sentiment == 'negative']['text'])

            # Remove duplicates sentences
            pos1 = pos1 - pos2
            neg1 = neg1 - neg2

        # If you would like to don't use separate files use this lines below
        # pos1 = [{'sentiment': 'positive', 'text': x} for x in pos1]
        # neg1 = [{'sentiment': 'negative', 'text': x} for x in neg1]
        # df1 = pd.DataFrame(pos1 + neg1)

        df1 = pd.DataFrame(list(pos1))
        df2 = pd.DataFrame(list(neg1))
        df1.to_csv(output_file_name_pos.format(i), mode='a', index=False)
        df2.to_csv(output_file_name_neg.format(i), mode='a', index=False)
        if auto_remove_files:
            os.remove(input_file_name.format(i))


In [6]:
def count_files_in_dir(input_file_path_with_name):
    counted_files_in_folder = [x for x in os.listdir(os.path.split(input_file_path_with_name)[0]) if
                               os.path.split(input_file_path_with_name)[-1].replace("{", '').replace("}", '') in x
                               and os.path.split(input_file_path_with_name)[-1].replace("{", '').replace("}", '') ==
                               re.sub(r'\d', '', x)].__len__()

    return counted_files_in_folder

In [7]:
def count_rows_in_files(input_file, files: int):
    out = []
    out_sum = 0
    for i in range(files):
        df = pd.read_csv(input_file.format(i))
        out.append(df.__len__())
    for elem in out:
        out_sum += elem
    return out_sum

In [ ]:
if __name__ == '__main__':
    # Declare re sizer function
    re_sizer = ReSizerFiles.re_sizer
    #
    tt = time.time()
    #

    # Slicer variables
    input_path_to_large_file = r'D:\s_a\All_Amazon_Review.json'
    output_file_path_post_slicer = r'D:\s_a\file_{}'

    # .json to .csv formatter
    cores = 4
    input_file_path = r'D:\s_a\file_{}'

    # Duplication remover variables
    input_file_name = r'D:\s_a\file_large_{}'
    output_file_name_neg = r'D:\s_a\file_large_n_{}'
    output_file_name_pos = r'D:\s_a\file_large_p_{}'

    # Re sizer variables
    output_positive_files = r'D:\s_a\p_{}'
    output_negative_files = r'D:\s_a\n_{}'

    slicer(input_path_to_large_file, output_file_path_post_slicer, 1200000)

    files_name = [input_file_path.format(x) for x in range(count_files_in_dir(input_file_path))]
    with multiprocessing.Pool(cores) as p:
        p.map(json_to_csv, files_name)

    re_sizer(input_file_name=input_file_path,
             output_file_name=input_file_name,
             max_length_of_single_output=8000000,
             auto_file_remover=True,
             auto_detect_files_in_dir=True,
             files_in_folder=None)

    duplicate_remover(input_file_name,
                      output_file_name_neg,
                      output_file_name_pos,
                      files_in_folder=count_files_in_dir(input_file_name),
                      auto_remove_files=True)

    re_sizer(input_file_name=output_file_name_neg,
             output_file_name=output_negative_files,
             max_length_of_single_output=1000000,
             auto_file_remover=True,
             auto_detect_files_in_dir=True,
             files_in_folder=None)

    re_sizer(input_file_name=output_file_name_pos,
             output_file_name=output_positive_files,
             max_length_of_single_output=1000000,
             max_amount_output_of_files=count_files_in_dir(output_negative_files),
             total_length_of_output=count_rows_in_files(output_negative_files,
                                                        count_files_in_dir(output_negative_files)),
             auto_file_remover=True,
             auto_detect_files_in_dir=True,
             files_in_folder=None)

    o = time.time() - tt
    print(o)
